In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.spatial import distance
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, LineString
import pandas as pd
#from src.generate_traffic_data import clean_carla, spawn_scene, run_scene
import carla
import os
os.getcwd()
from datetime import datetime
import time
import random
from tqdm import tqdm
import networkx as nx
import numpy as np
import pickle

os.chdir('../..')
os.getcwd()

#from graph_creator.MapGraph import MapGraph
#from graph_creator.ActorGraph import ActorGraph

from graph_creator.plot_graphs import plot_lane_map_advanced, add_actors_to_map, add_actor_edges_to_map

### Start comparing graphs:

In [3]:
from glob import glob
import pickle
import networkx as nx
import random
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

graph_paths = glob("/home/tmuehlen/repos/graph_coverage/actor_graphs/carla/*.pkl")
len(graph_paths)

7175

In [5]:
random.sample(list(range(len(graph_paths))), 10)

[5454, 603, 2441, 49, 2542, 765, 5193, 451, 2453, 137]

In [6]:
# Define a random set of coverage situations with 3 nodes each
# This still needs to be replaced by more strategically selected graphs

#idxs = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
idxs = random.sample(list(range(len(graph_paths))), 10)

coverage_graphs = {}
cov_data = {}

for idx in idxs:
    graph_path = graph_paths[0]
    with open(graph_path, "rb") as file:
        ag_nx = pickle.load(file)
    #random.sample(ag_nx.nodes, 3)
    coverage_graphs[idx] = ag_nx.subgraph(random.sample(list(ag_nx.nodes), 3))
    cov_data[idx] = []

cov_data["degree"] = []
cov_data["density"] = []
cov_data["diameter"] = []


In [7]:
for graph_path in graph_paths:
    with open(graph_path, "rb") as file:
        ag_nx = pickle.load(file)

    for key in coverage_graphs.keys():
        GM = nx.algorithms.isomorphism.DiGraphMatcher(
            ag_nx, coverage_graphs[key],
            node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
            # node_match=nx.algorithms.isomorphism.categorical_node_match(['actor_type', 'lane_change'], [None, None]),
            edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
        )
        cov_data[key].append(GM.subgraph_is_isomorphic())
    
    cov_data["degree"].append(sum(dict(ag_nx.degree()).values()) / len(ag_nx.nodes()))
    cov_data["density"].append(nx.density(ag_nx))
    cov_data["diameter"].append(nx.diameter(ag_nx))

cov_data_df = pd.DataFrame(cov_data)
cov_data_df.mean()

4060        0.354983
5025        0.307456
3954        0.354983
54          0.307456
5907        0.354983
3291        0.307456
6417        0.307456
5274        0.307456
1157        0.307456
2805        0.307456
degree      2.705190
density     0.669820
diameter    3.215889
dtype: float64

In [8]:
list(cov_data_df.columns[:10])


[4060, 5025, 3954, 54, 5907, 3291, 6417, 5274, 1157, 2805]

In [9]:
cov_data_df.groupby(list(cov_data_df.columns[:11]), as_index=False).size()

,4060,5025,3954,54,5907,3291,6417,5274,1157,2805,degree,size
0,False,False,False,False,False,False,False,False,False,False,2.000000,3257
1,False,False,False,False,False,False,False,False,False,False,2.666667,1228
2,False,True,False,True,False,True,True,True,True,True,3.000000,111
3,False,True,False,True,False,True,True,True,True,True,3.200000,29
4,False,True,False,True,False,True,True,True,True,True,3.333333,3
...,...,...,...,...,...,...,...,...,...,...,...,...
188,True,True,True,True,True,True,True,True,True,True,5.191489,1
189,True,True,True,True,True,True,True,True,True,True,5.254902,1
190,True,True,True,True,True,True,True,True,True,True,5.307692,1
191,True,True,True,True,True,True,True,True,True,True,5.333333,1


In [ ]:
# sns.pairplot(data=cov_data_df)
# plt.show()


# Notebook junkyard